In [ ]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from shutil import copyfile
# copyfile(src="../input/functions/utils.py", dst="../working/utils.py")
from utils import *
import zipfile
import os
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms
from torchvision import datasets

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable
import PIL.Image as Image
import torch
from tqdm import tqdm
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64
from tqdm import tqdm

is_trained=True

def create_download_link(title="Download CSV file", filename="experiment/kaggle.csv"):
    df = pd.read_csv(filename)
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


def sample_image():
    bird_classes = os.listdir('../input/bird_dataset/bird_dataset/train_images/')
    sample_class = np.random.choice(bird_classes)
    training_path = '../input/bird_dataset/bird_dataset/train_images/'
    sample_image = np.random.choice(os.listdir(training_path+sample_class))
    path_sample_image = training_path + sample_class + '/' + sample_image
    x = plt.imread(path_sample_image)
    return x



def train(epoch, model, train_loader, record_loss=True):
    """
        Train the global defined model.
        Warnings:
            model: is a global variable here
        Input:
            epoch: The index number of the epoch (This is NOT the number of epochs)
            record_loss: Record the loss evolution for the model
    """
    # loss recording
    loss_evolution = []

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        criterion = torch.nn.CrossEntropyLoss(reduction='elementwise_mean')
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            loss_value = loss.data.item()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss_value))
        if record_loss:
            loss_evolution.append(loss_value)

    if record_loss:
        return loss_evolution

def get_model(path_experiment, number=-1):
    if number < 0:
        model_names = os.listdir(path_experiment)
        model_numbers = []
        for name in model_names:
            regex = r'(\d+)\.pth'
            pattern = re.compile(regex)
            match = re.search(pattern, name)
            if match is not None:
                model_numbers.append(int(match.group(1)))
        number = np.max(model_numbers)
    return f'./experiment/model_{number}.pth'


def validation(model, val_loader):
    """
        Compute validation score
        Warnings:
            As in train(), the model is a global variable definition
        Input:
        Output:
            validation_loss <float>
            accuracy <float>

    """
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        criterion = torch.nn.CrossEntropyLoss(reduction='elementwise_mean')
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        accuracy))

    return validation_loss, accuracy

In [ ]:
# bird_classes = os.listdir('../input/mva-recvis-2018/bird_dataset/bird_dataset/train_images/')
bird_classes = os.listdir('../input/bird_dataset/bird_dataset/train_images/')
sample_class = np.random.choice(bird_classes)
training_path = '../input/bird_dataset/bird_dataset/train_images/'

# parameters
n_classes = len(bird_classes)

## Model

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

from torchvision.models import alexnet
model = alexnet(pretrained=False, num_classes=n_classes)

## main.py

In [ ]:
args_data = '../input/bird_dataset/bird_dataset/'
args_batch_size = 100
args_epochs = 164
args_lr = 0.01
args_momentum = 0.5

args_seed = 1
args_log_interval = 10
args_experiment = 'experiment'

use_cuda = torch.cuda.is_available()
if use_cuda:
    model.cuda()
torch.manual_seed(args_seed)

# Create experiment folder
if not os.path.isdir(args_experiment):
    os.makedirs(args_experiment)
    
print('Folder:')
for folder_ in os.listdir():
    print('    /' + folder_)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args_data + '/train_images',
                         transform=data_transforms),
    batch_size=args_batch_size, shuffle=True, num_workers=0)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args_data + '/val_images',
                         transform=data_transforms),
    batch_size=args_batch_size, shuffle=False, num_workers=0)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)
# optimizer = torch.optim.Adam(model.parameters(), lr=args_lr)

In [ ]:
# Go through epochs and record loss on training set and accuracy on
train_loss_evolution = []
valid_loss_evolution = []
valid_accuracy_evolution = []

if not is_trained:
    for epoch in range(1, args_epochs + 1):
        loss_evolution = train(epoch, model, train_loader, record_loss=True)
        validation_loss, accuracy = validation(model, val_loader)
        # Record
        train_loss_evolution.append(loss_evolution)
        valid_loss_evolution.append(validation_loss)
        valid_accuracy_evolution.append(accuracy)

        # Store the last 2 models
        if epoch > args_epochs-2:
            model_file = args_experiment + '/model_' + str(epoch) + '.pth'
            torch.save(model.state_dict(), model_file)
            print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

#### Visualise loss and accuracy evolutions

In [ ]:
if not is_trained:
    plt.figure(figsize=(15,15))

    plt.subplot('311')
    plt.title('Evolution of the training loss')
    plt.plot(np.mean(np.array(train_loss_evolution), axis=1), alpha=0.4, label='mean')
    plt.plot(np.mean(np.array(train_loss_evolution), axis=1) + 3*np.std(np.array(train_loss_evolution), axis=1), 'rs--', alpha=0.4 ,label='std_bound')
    plt.plot(np.mean(np.array(train_loss_evolution), axis=1) - 3*np.std(np.array(train_loss_evolution), axis=1), 'rs--', alpha=0.4)
    plt.boxplot(np.array(train_loss_evolution).T)
    plt.xlabel('epochs')
    plt.ylabel('validation loss')
    plt.legend()
    plt.grid()

    plt.subplot('312')
    plt.title('Validation loss evolution')
    plt.plot(np.array(valid_loss_evolution), 'xk')
    plt.plot(np.array(valid_loss_evolution), 'k--', alpha=0.4)

    plt.grid()

    plt.subplot('313')
    plt.title('Validation accuracy evolution')
    plt.plot(np.array(valid_accuracy_evolution), 'xk')
    plt.plot(np.array(valid_accuracy_evolution), 'k--', alpha=0.4)
    plt.grid()

    plt.show()

## Evaluate

In [ ]:
args_outfile = os.path.join(args_experiment, 'kaggle.csv')
args_model = get_model(args_experiment)
print(f"Loading {args_model}")

state_dict = torch.load(args_model)
# model = Net()
model.load_state_dict(state_dict)
model.eval()
if use_cuda:
    model.cuda()

test_dir = args_data + '/test_images/mistery_category'


output_file = open(args_outfile, "w")
output_file.write("Id,Category\n")
for f in tqdm(os.listdir(test_dir)):
    if 'jpg' in f:
        data = data_transforms(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if use_cuda:
            data = data.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()

print("Succesfully wrote " + args_outfile + ', you can upload this file to the kaggle competition website')

# Check if output_file is available
print('kaggle.csv' in os.listdir('./experiment/'))
print(os.listdir('experiment'))

# create a link to download the dataframe
create_download_link("experiment/kaggle.csv")

